# Genetic attribution overview

In this notebook we overview the [Genetic Engineering Attrinution competition in Driven Data](https://www.drivendata.org/competitions/63/genetic-engineering-attribution/).   

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from collections import OrderedDict

## Loading data

In [2]:
%%time
data_dir = "/home/rio/genetic_engineering_attribution/data"
data_list = os.listdir(data_dir)
data_list

CPU times: user 576 µs, sys: 49 µs, total: 625 µs
Wall time: 401 µs


['train_labels.csv',
 'train_values.csv',
 'test_values.csv',
 'split_labs',
 'submission_format_3TFRxH6.csv']

#### Training features

In [3]:
%%time
train_features_path = os.path.join(data_dir,data_list[1])
train_features = pd.read_csv(train_features_path)
print("shape of train_features: ", train_features.shape)
train_features.head()

shape of train_features:  (63017, 41)
CPU times: user 2.25 s, sys: 168 ms, total: 2.42 s
Wall time: 2.41 s


,sequence_id,sequence,bacterial_resistance_ampicillin,bacterial_resistance_chloramphenicol,bacterial_resistance_kanamycin,bacterial_resistance_other,bacterial_resistance_spectinomycin,copy_number_high_copy,copy_number_low_copy,copy_number_unknown,...,species_budding_yeast,species_fly,species_human,species_mouse,species_mustard_weed,species_nematode,species_other,species_rat,species_synthetic,species_zebrafish
0,9ZIMC,CATGCATTAGTTATTAATAGTAATCAATTACGGGGTCATTAGTTCA...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5SAQC,GCTGGATGGTTTGGGACATGTGCAGCCCCGTCTCTGTATGGAGTGA...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,E7QRO,NNCCGGGCTGTAGCTACACAGGGCGGAGATGAGAGCCCTACGAAAG...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,CT5FP,GCGGAGATGAAGAGCCCTACGAAAGCTGAGCCTGCGACTCCCGCAG...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7PTD8,CGCGCATTACTTCACATGGTCCTCAAGGGTAACATGAAAGTGATCC...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Training targets

In [4]:
%%time
train_targets_path = os.path.join(data_dir,data_list[0])
train_targets = pd.read_csv(train_targets_path)
print("shape of train_targets: ", train_targets.shape)
train_targets.head()

shape of train_targets:  (63017, 1315)
CPU times: user 7.36 s, sys: 408 ms, total: 7.77 s
Wall time: 7.77 s


,sequence_id,00Q4V31T,012VT4JK,028IO5W2,03GRNN7N,03Y3W51H,09MQV1TY,0A4AHRCT,0A9M05NC,0B9GCUVV,...,ZQNGGY33,ZSHS4VJZ,ZT1IP3T6,ZU6860XU,ZU6TVFFU,ZU75P59K,ZUI6TDWV,ZWFD8OHC,ZX06ZDZN,ZZJVE4HO
0,9ZIMC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5SAQC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,E7QRO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,CT5FP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7PTD8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Exploratory Data analysis

## Preprocessing

#### One-hot encoding DNA sequences

In [23]:
def dna2dict(dna_seq):
    """
    DNA bases are ordered in tuples alphabetically as (A,C,G,N,T)
    """
    dna_dict = OrderedDict()
    dna_seq = dna_seq.upper()
    for ix,b in enumerate(dna_seq):
        dna_dict[ix] = (int(b=="A"),int(b=="C"),int(b=="G"),int(b=="N"),int(b=="T"))
    return dna_dict
        

In [24]:
dna_dict = dna2dict(train_features.sequence.values[1])
dna_dict

OrderedDict([(0, (0, 0, 1, 0, 0)),
             (1, (0, 1, 0, 0, 0)),
             (2, (0, 0, 0, 0, 1)),
             (3, (0, 0, 1, 0, 0)),
             (4, (0, 0, 1, 0, 0)),
             (5, (1, 0, 0, 0, 0)),
             (6, (0, 0, 0, 0, 1)),
             (7, (0, 0, 1, 0, 0)),
             (8, (0, 0, 1, 0, 0)),
             (9, (0, 0, 0, 0, 1)),
             (10, (0, 0, 0, 0, 1)),
             (11, (0, 0, 0, 0, 1)),
             (12, (0, 0, 1, 0, 0)),
             (13, (0, 0, 1, 0, 0)),
             (14, (0, 0, 1, 0, 0)),
             (15, (1, 0, 0, 0, 0)),
             (16, (0, 1, 0, 0, 0)),
             (17, (1, 0, 0, 0, 0)),
             (18, (0, 0, 0, 0, 1)),
             (19, (0, 0, 1, 0, 0)),
             (20, (0, 0, 0, 0, 1)),
             (21, (0, 0, 1, 0, 0)),
             (22, (0, 1, 0, 0, 0)),
             (23, (1, 0, 0, 0, 0)),
             (24, (0, 0, 1, 0, 0)),
             (25, (0, 1, 0, 0, 0)),
             (26, (0, 1, 0, 0, 0)),
             (27, (0, 1, 0, 0, 0)),
  

In [25]:
len(dna_dict)

456

In [26]:
list(dna_dict.values())

[(0, 0, 1, 0, 0),
 (0, 1, 0, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0),
 (0, 0, 1, 0, 0),
 (1, 0, 0, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0),
 (0, 0, 1, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 0, 0, 1),
 (0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0),
 (0, 0, 1, 0, 0),
 (0, 0, 1, 0, 0),
 (1, 0, 0, 0, 0),
 (0, 1, 0, 0, 0),
 (1, 0, 0, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0),
 (0, 1, 0, 0, 0),
 (1, 0, 0, 0, 0),
 (0, 0, 1, 0, 0),
 (0, 1, 0, 0, 0),
 (0, 1, 0, 0, 0),
 (0, 1, 0, 0, 0),
 (0, 1, 0, 0, 0),
 (0, 0, 1, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 1, 0, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 1, 0, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0),
 (0, 0, 0, 0, 1),
 (1, 0, 0, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0),
 (0, 0, 1, 0, 0),
 (1, 0, 0, 0, 0),
 (0, 0, 1, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0),
 (1, 0, 0, 0, 0),
 (0, 1, 0, 0, 0),
 (1, 0, 0, 0, 0),
 (0, 0, 0, 0, 1),
 (1, 0, 0, 0, 0),
 (0, 0, 1, 0, 0),
 (1, 0, 0, 0, 0),
 (0, 0, 1, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0),
 (0, 0, 0,

#### One-hot encoded DNA sequences as dataframes

In [29]:
def dna2df(dna_seq,seq_id=None, lab_id = None):
    dna_dict = dna2dict(dna_seq)
    dna_df = pd.DataFrame(dna_dict.values(),columns=("A","C","G","N","T"))
    if seq_id:
        columns = ["sequence_id"] + list(dna_df.columns)
        dna_df["sequence_id"] = [seq_id]*len(dna_df)
        dna_df = dna_df.loc[:,columns]
    if lab_id:
        columns = ["lab_id"] + list(dna_df.columns)
        dna_df["lab_id"] = [lab_id]*len(dna_df)
        dna_df = dna_df.loc[:,columns]
    return dna_df


In [32]:
dna_seq = train_features.sequence.values[0]
seq_id = train_features.sequence_id.values[0]
lab_id = train_targets.columns[train_targets.iloc[0,:]==1][0]
dna_df = dna2df(dna_seq=dna_seq,seq_id=seq_id,lab_id=lab_id)
dna_df

,lab_id,sequence_id,A,C,G,N,T
0,RYUA3GVO,9ZIMC,0,1,0,0,0
1,RYUA3GVO,9ZIMC,1,0,0,0,0
2,RYUA3GVO,9ZIMC,0,0,0,0,1
3,RYUA3GVO,9ZIMC,0,0,1,0,0
4,RYUA3GVO,9ZIMC,0,1,0,0,0
...,...,...,...,...,...,...,...
7146,RYUA3GVO,9ZIMC,1,0,0,0,0
7147,RYUA3GVO,9ZIMC,0,1,0,0,0
7148,RYUA3GVO,9ZIMC,0,1,0,0,0
7149,RYUA3GVO,9ZIMC,0,0,1,0,0


In [31]:
lab_id

Index(['RYUA3GVO'], dtype='object')

#### Generating separate csvs for each lab

In [5]:
%%time
split_labs_dir = os.path.join(data_dir,"split_labs")
if not os.path.isdir(split_labs_dir):
    os.mkdir(split_labs_dir)
unique_labs = train_targets.columns[1:]
pbar = tqdm(unique_labs)
for lab in tqdm(unique_labs):
    lab_path = os.path.join(split_labs_dir,lab)
    if not os.path.isdir(lab_path):
        pbar.set_description(f"Generating directory {lab_path}")
        os.mkdir(lab_path)
    else:
        pbar.set_description(f"Directory {lab_path} already exists. Skipping.")


  0%|          | 0/1314 [00:00<?, ?it/s]
Directory /home/rio/genetic_engineering_attribution/data/split_labs/3D9CMQ4V already exists. Skipping.:   0%|          | 0/1314 [00:00<?, ?it/s]
Directory /home/rio/genetic_engineering_attribution/data/split_labs/68OY1RK5 already exists. Skipping.:   0%|          | 0/1314 [00:00<?, ?it/s]
Directory /home/rio/genetic_engineering_attribution/data/split_labs/9MZBKXJF already exists. Skipping.:   0%|          | 0/1314 [00:00<?, ?it/s]
Directory /home/rio/genetic_engineering_attribution/data/split_labs/CENOJ84D already exists. Skipping.:   0%|          | 0/1314 [00:00<?, ?it/s]
Directory /home/rio/genetic_engineering_attribution/data/split_labs/FPH5H8JT already exists. Skipping.:   0%|          | 0/1314 [00:00<?, ?it/s]
Directory /home/rio/genetic_engineering_attribution/data/split_labs/IPVYEI8G already exists. Skipping.:   0%|          | 0/1314 [00:00<?, ?it/s]
Directory /home/rio/genetic_engineering_attribution/data/split_labs/LL11R5T6 already exis

CPU times: user 1.17 s, sys: 233 ms, total: 1.41 s
Wall time: 1.17 s


In [ ]:
split_labs_dir = os.path.join(data_dir,"split_labs")
def split_labs(train_targets,savedir=split_labs_dir):
    